# Segmenting and Clustering Neighborhoods in Toronto

In this notebook, we will explore, segment, and cluster the neighborhoods in the city of Toronto. We will end up with groups of similar neighborhoods based on the categories of the venues in each neighborhood. 

For the Toronto neighborhood data, a Wikipedia page exists that has information we need to explore and cluster the neighborhoods in Toronto. It contains the name of each neighborhood along with the borough and the postal code of the neighborhood.

## Part 1: Scraping the Data

### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import folium
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
pd.set_option('max_rows', 140)

### Reading the table from the Wikipedia page

In [3]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = dfs[0]

In [4]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
df.shape

(288, 3)

### Making small modifications on the column names

In [6]:
df.columns = ['PostalCode', 'Borough', 'Neighborhood']

### Removing records where the borough is "not assigned"

In [7]:
df = df[df.Borough != 'Not assigned']

In [8]:
df.shape

(211, 3)

### Merging records where multiple neighborhoods share the same borough

For example, there are two neighborhoods (Harbourfront and Regent Park) that share the same borough (Downtown Toronto) as shown below:

In [9]:
df[df.PostalCode == 'M5A']

,PostalCode,Borough,Neighborhood
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


We will merge records where multiple neighborhoods share the same borough:

In [10]:
df = (df.groupby(['PostalCode', 'Borough'])['Neighborhood']
      .apply(lambda x: "{}".format(', '.join(x))).reset_index())

Now let's take a look at the record of Downtown Toronto borough:

In [11]:
df[df.PostalCode == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


We can see that the previous two records were merged into one and the neighborhood names got concatenated.

### Dealing with the case where the neighborhood is "not assigned"

For some records in the table, there is a valid value for the borough but the neighborhood has the value "Not assigned" as shown below:

In [12]:
df[df.Neighborhood.str.contains('Not assigned')]

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In this case, we will give the borough name to the neighborhood:

In [13]:
df.loc[df.Neighborhood.str.contains('Not assigned'), 'Neighborhood'] = \
    df.loc[df.Neighborhood.str.contains('Not assigned'), 'Borough']

Now let's take a look at the previous record:

In [14]:
df[df.Borough == "Queen's Park"]

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


We can see that the neighborhood got the same name as the borough.

### The shape of the final neighborhood table

In [15]:
df.shape

(103, 3)

## Part 2: Getting Latitude and Longitude Coordinates of the Neighborhoods

In [16]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Reading a file that contains the coordinates of each borough

In [17]:
lat_lng_df = pd.read_csv('https://cocl.us/Geospatial_data')

In [18]:
lat_lng_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Adding langitude and latitude to our dataframe

In [19]:
df.reset_index(drop=True, inplace=True)
df['Latitude'] = -99999.9
df['Longitude'] = -99999.9

for i in range(df.shape[0]):
    postalcode = df.loc[i, 'PostalCode']
    lat = lat_lng_df.loc[lat_lng_df['Postal Code'] == postalcode, 'Latitude'].squeeze()
    lng = lat_lng_df.loc[lat_lng_df['Postal Code'] == postalcode, 'Longitude'].squeeze()
    df.loc[i, 'Latitude'] = lat
    df.loc[i, 'Longitude'] = lng    

In [20]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part 3: Clustering of the Neighborhoods

### Creating a map of Toronto with neighborhoods superimposed on top

In [21]:
# Toronto coordinates
latitude = 43.6532
longitude = -79.3832

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add neighborhood markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], 
                                           df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#333333',
        fill=True,
        fill_color='#ff5349',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Getting neighborhood data from Foursquare using its API

In [22]:
# Foursquare credentials
CLIENT_ID = 'A1VPXW0G4VZLCKIMZUOGCWBXBGIY5UP21CRGT0R4OJXJOWBQ' 
CLIENT_SECRET = 'FGC4DCS0JRFEC25IFE1NC0AFZM2KDTVRRFUK13VFVNG4DHXC' 
VERSION = '20180605'

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    """
    A function that retrieves information about venues in each neighborhood.
    It takes as input the names of the neighborhoods, their latitudes and longitudes.
    It returns a dataframe with information about each neighborhood and its venues.
    """
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = ('https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}'
               '&v={}&ll={},{}&radius={}&limit={}'
               .format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT))
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, v['venue']['name'], 
                             v['venue']['location']['lat'], 
                             v['venue']['location']['lng'],  
                             v['venue']['categories'][0]['name']) 
                            for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 
                             'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)


tor_venues = getNearbyVenues(names=df['Neighborhood'],
                             latitudes=df['Latitude'],
                             longitudes=df['Longitude'])

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [25]:
tor_venues.shape

(2271, 7)

In [26]:
tor_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


### How many venues were returned for each neighborhood?

In [27]:
tor_venues.groupby('Neighborhood').size().to_frame('Venue count')

,Venue count
Neighborhood,
"Adelaide, King, Richmond",100
Agincourt,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10
"Alderwood, Long Branch",9
"Bathurst Manor, Downsview North, Wilson Heights",18
Bayview Village,4
"Bedford Park, Lawrence Manor East",25
Berczy Park,55


We notice that there are 99 neighborhoods in the table while we had 103 neighborhoods before we got the data from Foursquare. This means that Foursquare didn't return any venue for 4 neighborhoods. These 4 neighborhoods are:

In [28]:
excluded_neighborhoods = set(df['Neighborhood']).difference(tor_venues['Neighborhood'])
excluded_neighborhoods

{'Islington Avenue', 'Newtonbrook, Willowdale', 'Upper Rouge'}

### Performing one-hot on the venue category variable

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
# we used "Neighborhood_" instead of just "Neighborhood" because
# there is a venue category  called "Neighborhood"
toronto_onehot['Neighborhood_'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood_,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
toronto_onehot.shape

(2271, 274)

### Grouping rows by neighborhood and taking the mean of the frequency of occurrence of each category for each neighborhood

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighborhood_').mean().reset_index()
toronto_grouped.head()

,Neighborhood_,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,...,0.0,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [32]:
toronto_grouped.shape

(100, 274)

### The most common categories for each neighborhood

In [33]:
def return_most_common_venues(row, num_top_cat):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_cat]


num_top_cat = 5
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood_']
for ind in np.arange(num_top_cat):
    try:
        columns.append('{}{} Most Common Category'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Category'.format(ind+1))

# create a new dataframe
neighborhoods_categories_sorted = pd.DataFrame(columns=columns)
neighborhoods_categories_sorted['Neighborhood_'] = toronto_grouped['Neighborhood_']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_categories_sorted.iloc[ind, 1:] = return_most_common_venues(
        toronto_grouped.iloc[ind, :], num_top_cat)

neighborhoods_categories_sorted.head()

,Neighborhood_,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Cosmetics Shop
1,Agincourt,Lounge,Breakfast Spot,Skating Rink,Clothing Store,Yoga Studio
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Sculpture Garden,Yoga Studio,Drugstore
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fast Food Restaurant,Liquor Store,Fried Chicken Joint,Sandwich Place
4,"Alderwood, Long Branch",Pizza Place,Gym,Dance Studio,Pharmacy,Coffee Shop


### Clustering

Now we apply K-means clustering on the dataframe stored in `toronto_grouped` variable which includes the relative frequency of each venue-category for each neighborhood.

In [34]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood_', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [36]:
# add clustering labels
neighborhoods_categories_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df.rename(columns={'Neighborhood': 'Neighborhood_'}).copy()
toronto_merged = toronto_merged[~toronto_merged['Neighborhood_'].isin(excluded_neighborhoods)]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_categories_sorted.set_index('Neighborhood_'), on='Neighborhood_')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood_,Latitude,Longitude,Cluster Labels,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,Bar,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Intersection,Breakfast Spot,Pizza Place,Rental Car Location,Electronics Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Yoga Studio,Discount Store,Dog Run
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Bakery,Hakka Restaurant,Bank,Caribbean Restaurant,Athletics & Sports


### Creating a map that shows the neighborhoods and their clusters
We will create a map that shows a marker for each neighborhood; the color of the marker represents the cluster of the neighborhood.

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], 
                                  toronto_merged['Neighborhood_'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining clusters
Let's see the neighborhoods in each of the five clusters:

#### Cluster 1

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category
1,Scarborough,0,Bar,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run
2,Scarborough,0,Intersection,Breakfast Spot,Pizza Place,Rental Car Location,Electronics Store
3,Scarborough,0,Coffee Shop,Korean Restaurant,Yoga Studio,Discount Store,Dog Run
4,Scarborough,0,Bakery,Hakka Restaurant,Bank,Caribbean Restaurant,Athletics & Sports
5,Scarborough,0,Playground,Grocery Store,Yoga Studio,Dumpling Restaurant,Diner
6,Scarborough,0,Discount Store,Department Store,Hobby Shop,Convenience Store,Coffee Shop
7,Scarborough,0,Bakery,Fast Food Restaurant,Intersection,Bus Station,Bus Line
8,Scarborough,0,American Restaurant,Movie Theater,Motel,Event Space,Ethiopian Restaurant
9,Scarborough,0,College Stadium,Skating Rink,General Entertainment,Café,Empanada Restaurant
10,Scarborough,0,Indian Restaurant,Vietnamese Restaurant,Light Rail Station,Latin American Restaurant,Chinese Restaurant


#### Cluster 2

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category
0,Scarborough,1,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run


#### Cluster 3

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category
14,Scarborough,2,Park,Playground,Sculpture Garden,Yoga Studio,Drugstore
20,North York,2,Park,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run
23,North York,2,Park,Bank,Convenience Store,Bar,Yoga Studio
25,North York,2,Food & Drink Shop,Park,Yoga Studio,Discount Store,Dog Run
44,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Drugstore
50,Downtown Toronto,2,Park,Playground,Trail,Building,Ethiopian Restaurant
74,York,2,Park,Women's Store,Market,Fast Food Restaurant,Yoga Studio
90,Etobicoke,2,River,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant
98,York,2,Park,Convenience Store,Yoga Studio,Eastern European Restaurant,Discount Store


#### Cluster 4

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category
63,Central Toronto,3,Garden,Drugstore,Dim Sum Restaurant,Diner,Discount Store


#### Cluster 5

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category
94,Etobicoke,4,Bank,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run
